In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
Sflow = pd.read_csv("SFlow_Data_lab4.csv",encoding = "ISO-8859-1")
print("Data type : ", type(Sflow))
print("Data dims : ", Sflow.shape)


Data type :  <class 'pandas.core.frame.DataFrame'>
Data dims :  (69370, 21)


In [76]:
print(Sflow.dtypes)


Type                    object
sflow_agent_address     object
inputPort                int64
outputPort               int64
src_MAC                 object
dst_MAC                 object
ethernet_type           object
in_vlan                float64
out_vlan                 int64
src_IP                  object
dst_IP                  object
IP_protocol              int64
ip_tos                  object
ip_ttl                   int64
src_IP_port            float64
dst_IP_port              int64
tcp_flags               object
packet_size              int64
IP_size                  int64
sampling_rate            int64
Unnamed: 20            float64
dtype: object


In [77]:
Sflow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69370 entries, 0 to 69369
Data columns (total 21 columns):
Type                   69370 non-null object
sflow_agent_address    69370 non-null object
inputPort              69370 non-null int64
outputPort             69370 non-null int64
src_MAC                69370 non-null object
dst_MAC                69370 non-null object
ethernet_type          69370 non-null object
in_vlan                69370 non-null float64
out_vlan               69370 non-null int64
src_IP                 69370 non-null object
dst_IP                 69370 non-null object
IP_protocol            69370 non-null int64
ip_tos                 69370 non-null object
ip_ttl                 69370 non-null int64
src_IP_port            69370 non-null float64
dst_IP_port            69370 non-null int64
tcp_flags              69370 non-null object
packet_size            69370 non-null int64
IP_size                69370 non-null int64
sampling_rate          69370 non-null int6

# EXERCISE 4A: TOP TALKERS AND LISTENERS# 

## Top 5 Talkers. (ie sender nodes)

In [78]:

scr_IP_grouped = Sflow[['src_IP','packet_size']].groupby(['src_IP'], sort=False).count()
scr_IP_grouped.nlargest(5,"packet_size")

,packet_size
src_IP,
193.62.192.8,3041
155.69.160.32,2975
130.14.250.11,2604
14.139.196.58,2452
140.112.8.139,2056


## Top 5 Listeners (ie receiving node)

In [79]:
dst_IP_grouped = Sflow[['dst_IP','packet_size']].groupby(['dst_IP'], sort=False).count()
dst_IP_grouped.nlargest(5,"packet_size")

,packet_size
dst_IP,
103.37.198.100,3841
137.132.228.15,3715
202.21.159.244,2446
192.101.107.153,2368
103.21.126.2,2056


# EXERCISE 4B: TRANSPORT PROTOCOL 

In [80]:
transport_grouped = Sflow[['IP_protocol','dst_IP' ]].groupby(['IP_protocol'], sort=False).count()
transport_grouped

,dst_IP
IP_protocol,
6,56064
17,9462
1,74
41,104
50,1698
0,1261
47,657
381,45
103,1


In [81]:
transport_grouped.nlargest(3,"dst_IP")


,dst_IP
IP_protocol,
6,56064
17,9462
50,1698


In [82]:
total_packet_number = Sflow["IP_protocol"].count()
IP_protocol_6_number = Sflow["IP_protocol"].loc[Sflow["IP_protocol"]==6].count()
IP_protocol_17_number = Sflow["IP_protocol"].loc[Sflow["IP_protocol"]==17].count()
IP_protocol_50_number = Sflow["IP_protocol"].loc[Sflow["IP_protocol"]==50].count()
print("percentage of IP_protocol 6 : %5.4f" % (IP_protocol_6_number/total_packet_number))
print("percentage of IP_protocol 17 : %5.4f" % (IP_protocol_17_number/total_packet_number))
print("percentage of IP_protocol 50 : %5.4f" % (IP_protocol_50_number/total_packet_number))

percentage of IP_protocol 6 : 0.8082
percentage of IP_protocol 17 : 0.1364
percentage of IP_protocol 50 : 0.0245


# EXERCISE 4C: APPLICATIONS PROTOCOL

In [83]:
dst_IP_port_number_grouped = Sflow[['dst_IP_port','packet_size']].groupby(['dst_IP_port'], sort=False).count()
dst_IP_port_number_grouped.nlargest(5,"packet_size")

,packet_size
dst_IP_port,
443,13423
80,2647
52866,2068
45512,1356
56152,1341


# EXERCISE 4D: TRAFFIC

In [84]:
#traffic_info = Sflow[['packet_size']]
traffic_info = Sflow[['IP_size']]
traffic_info

,IP_size
0,1500
1,52
2,1500
3,94
4,52
...,...
69365,1500
69366,1420
69367,64
69368,1245


In [85]:
total_traffic = traffic_info.sum()
print("Total traffic of the router with a sampling rate of 1000 : %5.2f (MB)" % (total_traffic*1000 / 1024 /1024))


Total traffic of the router with a sampling rate of 1000 : 61776.95 (MB)


# EXERCISE 4E: ADDITIONAL ANALYSIS

## Top 5 communication pair

In [86]:
communication_pair = Sflow.groupby(["src_IP", "dst_IP"]).size().reset_index(name = "Count")

frame_size = communication_pair.shape[0]
arr_src_IP = []
arr_dst_IP = []
count_array = []
drop_index_array = []
print("I'm trying to iterate all rows")
for index in range(frame_size):
    
    if (index in drop_index_array):
        continue
    
    src_ip = communication_pair.iloc[index]["src_IP"]
    dst_ip = communication_pair.iloc[index]["dst_IP"]
    count1 = communication_pair.iloc[index]["Count"]
    index2 = communication_pair.index[(communication_pair["dst_IP"] == src_ip) & (communication_pair["src_IP"] == dst_ip) & (~communication_pair.index.isin(drop_index_array))]
    #print(index2)
    if index2.empty:
        arr_src_IP.append(src_ip)
        arr_dst_IP.append(dst_ip)
        count_array.append(count1)
    else:
        drop_index_array.append(index2[0])
        count2 = communication_pair.iloc[index2[0]]["Count"]
        arr_src_IP.append(src_ip)
        arr_dst_IP.append(dst_ip)
        count_array.append(count1 + count2)
    
dataframe_3e = pd.DataFrame({"IP Address 1":arr_src_IP, "IP Address 2":arr_dst_IP, "number of packets":count_array})
print(dataframe_3e.sort_values(by = ["number of packets"], ascending=False).head(5))

I'm trying to iterate all rows
        IP Address 1     IP Address 2  number of packets
1237  137.132.228.15     193.62.192.8               4951
212   103.37.198.100    130.14.250.11               2842
1426   14.139.196.58  192.101.107.153               2368
1553   140.112.8.139     103.21.126.2               2056
1585   140.90.101.61     167.205.52.8               1752


## Visualization of communications between different IP hosts 

In [87]:
#visualize
import matplotlib.pyplot as plt
import networkx as nx
import scipy

scr__IP = Sflow['src_IP']
dst__IP = Sflow['dst_IP']

In [88]:
G = nx.Graph()
connect_pair = set()
nodes = set()

for i in range(Sflow.shape[0]):
    scrIP = scr__IP[i]
    dstIP = dst__IP[i]
    nodes.add(scrIP)
    nodes.add(dstIP)
    connect_pair.add((scrIP,dstIP))

In [ ]:
G.add_nodes_from(nodes)
G.add_edges_from(connect_pair)
plt.rcParams['figure.figsize'] = 150,120
p = nx.spring_layout(G,scale = 1.0, iterations = 50)

f=plt.figure()
nx.draw(G,p,ax=f.add_subplot(111),node_color = 'r',edge_color = 'g', with_labels = True)
f.savefig('4e.png')